<a href="https://colab.research.google.com/github/SatyaThakur/DL_Bike_share_demand_sklearn/blob/master/Bike_Share_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing TF 1.14.0

In [2]:
pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 95kB/s 
     |████████████████████████████████| 3.2MB 31.0MB/s 
     |████████████████████████████████| 491kB 44.5MB/s 
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


# Uploading Data File

In [3]:
from google.colab import files
files.upload();

Saving bikeshare_hour_data.csv to bikeshare_hour_data.csv


In [5]:
import pandas as pd
bike_df=pd.read_csv("bikeshare_hour_data.csv")
bike_df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


# Dummy Encoding the Categorical columns

In [6]:
bike_df.shape

(17379, 17)

In [7]:
season_dummies = pd.get_dummies(bike_df.season, prefix="season", drop_first=True)
mnth_dummies = pd.get_dummies(bike_df.mnth, prefix="mnth", drop_first=True)
weekday_dummies = pd.get_dummies(bike_df.weekday, prefix="weekday", drop_first=True)
weather_dummies = pd.get_dummies(bike_df.weathersit, prefix="weather", drop_first=True)

#appending dummies to the data frame
bike_df=pd.concat([bike_df,season_dummies,mnth_dummies,weather_dummies, weekday_dummies], axis=1)
bike_df.shape

(17379, 40)

In [8]:
#deleting dummy encoded variables from df
del bike_df['season']
del bike_df['mnth']
del bike_df['weekday']
del bike_df['weathersit']
del bike_df['casual']
del bike_df['registered']
del bike_df['instant']
del bike_df['dteday'] 
bike_df.shape

(17379, 32)

In [9]:
pd.options.display.max_columns=None
bike_df.head()

,yr,hr,holiday,workingday,temp,atemp,hum,windspeed,cnt,season_2,season_3,season_4,mnth_2,mnth_3,mnth_4,mnth_5,mnth_6,mnth_7,mnth_8,mnth_9,mnth_10,mnth_11,mnth_12,weather_2,weather_3,weather_4,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0,0,0,0,0.24,0.2879,0.81,0.0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0.22,0.2727,0.80,0.0,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,2,0,0,0.22,0.2727,0.80,0.0,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,3,0,0,0.24,0.2879,0.75,0.0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,4,0,0,0.24,0.2879,0.75,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


Converting cnt type to float64 from int64

In [0]:
bike_df['cnt'] = bike_df['cnt'].astype('float64')

defining target and features

In [11]:
target = bike_df['cnt']
bike_df.drop(['cnt'],1,inplace=True)
features = bike_df
print(target.shape)
print(features.shape)

(17379,)
(17379, 31)


In [12]:
target

0         16.0
1         40.0
2         32.0
3         13.0
4          1.0
         ...  
17374    119.0
17375     89.0
17376     90.0
17377     61.0
17378     49.0
Name: cnt, Length: 17379, dtype: float64

In [13]:
features

,yr,hr,holiday,workingday,temp,atemp,hum,windspeed,season_2,season_3,season_4,mnth_2,mnth_3,mnth_4,mnth_5,mnth_6,mnth_7,mnth_8,mnth_9,mnth_10,mnth_11,mnth_12,weather_2,weather_3,weather_4,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0,0,0,0,0.24,0.2879,0.81,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0.22,0.2727,0.80,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,2,0,0,0.22,0.2727,0.80,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,3,0,0,0.24,0.2879,0.75,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,4,0,0,0.24,0.2879,0.75,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,1,19,0,1,0.26,0.2576,0.60,0.1642,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0
17375,1,20,0,1,0.26,0.2576,0.60,0.1642,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0
17376,1,21,0,1,0.26,0.2576,0.60,0.1642,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
17377,1,22,0,1,0.26,0.2727,0.56,0.1343,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


# Input features are given in different scale and should be stadardized before feeding them into Neural Network model

In [0]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [15]:
feature_set = ss.fit_transform(features)
feature_set

array([[-1.0051343 , -1.67000398, -0.1721122 , ..., -0.40712398,
        -0.40865929,  2.43277449],
       [-1.0051343 , -1.52537422, -0.1721122 , ..., -0.40712398,
        -0.40865929,  2.43277449],
       [-1.0051343 , -1.38074446, -0.1721122 , ..., -0.40712398,
        -0.40865929,  2.43277449],
       ...,
       [ 0.99489192,  1.36722104, -0.1721122 , ..., -0.40712398,
        -0.40865929, -0.41105331],
       [ 0.99489192,  1.51185081, -0.1721122 , ..., -0.40712398,
        -0.40865929, -0.41105331],
       [ 0.99489192,  1.65648057, -0.1721122 , ..., -0.40712398,
        -0.40865929, -0.41105331]])

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(feature_set,target,test_size=0.20,random_state=1234)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(13903, 31)
(3476, 31)
(13903,)
(3476,)


# Build Model

In [0]:
from keras import Sequential
from keras import layers


In [0]:
model = Sequential()

In [0]:
model.add(layers.Dense(150,activation='relu',input_shape=(31,)))

In [0]:
model.add(layers.Dense(90,activation='relu'))
model.add(layers.Dense(60,activation='relu'))
model.add(layers.Dense(30,activation='relu'))
model.add(layers.Dense(1,activation='relu'))

In [0]:
model.compile(optimizer='adam', loss='mse')

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 150)               4800      
_________________________________________________________________
dense_2 (Dense)              (None, 90)                13590     
_________________________________________________________________
dense_3 (Dense)              (None, 60)                5460      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                1830      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 25,711
Trainable params: 25,711
Non-trainable params: 0
_________________________________________________________________


# Fit the model

In [0]:
import tensorflow as tf 

In [28]:
model.fit(x_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
13903/13903 [==============================] - 1s 61us/step - loss: 25179.0819
Epoch 2/100
13903/13903 [==============================] - 1s 50us/step - loss: 18698.0521
Epoch 3/100
13903/13903 [==============================] - 1s 57us/step - loss: 17113.4744
Epoch 4/100
13903/13903 [==============================] - 1s 52us/step - loss: 13528.7031
Epoch 5/100
13903/13903 [==============================] - 1s 50us/step - loss: 11444.4361
Epoch 6/100
13903/13903 [==============================] - 1s 49us/step - loss: 9828.2307
Epoch 7/100
13903/13903 [==============================] - 1s 51us/step - loss: 8498.3591
Epoch 8/100
13903/13903 [==============================] - 1s 51us/step - loss: 7172.0042
Epoch 9/100
13903/13903 [==============================] - 1s 49us/step - loss: 5958.1016
Epoch 10/100
13903/13903 [==============================] - 1s 55us/step - loss: 4777.6991
Epoch 11/100
13903/13903 [==============================] - 1s 50us/step - loss: 3889.9565
Ep

# Model Validation

In [30]:
pred = model.predict(x_test)
pred

array([[371.79572  ],
       [350.5037   ],
       [  3.5580819],
       ...,
       [272.97232  ],
       [ 94.36443  ],
       [ 88.88593  ]], dtype=float32)

In [0]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [33]:
y_test=np.array(y_test)
y_test

array([330., 351.,   3., ..., 238.,  99.,  83.])

# Error finding

In [34]:
RMSE = np.sqrt(mean_squared_error(y_test,pred))
print('RMSE of the model is: ', RMSE)

RMSE of the model is:  40.59927920769863
